# Libraries

In [1]:
# using conda library 'ontogpt_fork'; naming irrelevant as this was previously going to be used to work on a different project.

# increase cell-width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import re
import numpy as np
import os
import glob
from tqdm import tqdm # progress bar tracking

pd.set_option('display.max_rows', None)
pd.set_option('display.width', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

import json
import jsonschema
from jsonschema import validate

# prompts for OpenAI

import openai_secret_manager
import openai
import time
import random

import tiktoken
from typing import List, Tuple

from collections import Counter

# Load SciERC dataset
* http://nlp.cs.washington.edu/sciIE/
* annotation guideline: http://nlp.cs.washington.edu/sciIE/annotation_guideline.pdf

* SciERC is an annotated dataset of 500 scientific abstracts (multi-sentence).
* Each article has 3 files:
    * .txt being the raw text.
    * .xml being the post-processed article.
    * .ann being the annotations.

## Annotation guideline

Relation link can not go beyond sentence boundary.

We define 4 asymmetric relation types (Used-for, Feature-of, Hyponym-of, Part-of), together with 2 symmetric relation types (Compare, Conjunction).
B always points to A for asymmetric relations.

* Used-for: B is used for A, B models A, A is trained on B, B exploits A, A is based on B. E.g.
    * The TISPER system has been designed to enable many text applications.
    * Our method models user proficiency.
    * Our algorithms exploits local soothness.

* Feature-of: B belongs to A, B is a feature of A, B is under A domain. E.g.
    * prior knowledge of the model
    * genre-specific regularities of discourse structure
    * English text in science domain

* Hyponym-of: B is a hyponym of A, B is a type of A. E.g.
    * TUIT is a software library
    * NLP applications such as machine translation and language generation

* Part-of: B is a part of A... E.g.
    * The system includes two models: speech recognition and natural language understanding
    * We incorporate NLU module to the system.

* Compare: Symmetric relation (use blue to denote entity). Opposite of conjunction, compare two models/methods, or listing two opposing entities. E.g.
    * Unlike the quantitative prior, the qualitative prior is often ignored...

* Conjunction: Symmetric relation (use blue to denote entity). Function as similar role or use/incorporate
with. E.g.
    * obtained from human expert or knowledge base
    * NLP applications such as machine translation and language generation

Coreference:
* Anaphora and Cataphora:
    * We introduce a machine reading system... The system...
    * The prior knowledge include...Such knowledge can be applied to...

* Coreferring noun phrase:
    * We develop a part-of-speech tagging system...The POS tagger...

## Explore .ann files

In [2]:
# Directory path:
directory_path = './benchmarking/input/sciERC_raw'

# Get the list of .ann files in the directory
ann_files = glob.glob(os.path.join(directory_path, '*.ann'))

# Iterate over each .ann file
for ann_file in ann_files:
    # Load the .ann file
    with open(ann_file, 'r') as f:
        annotations = f.readlines()

    # Print the .ann file name
    print("Annotations in", ann_file)

    # Print the annotations
    for annotation in annotations:
        print(annotation.strip())

    print("-----------")

Annotations in ./benchmarking/input/sciERC_raw\A00-1024.ann
T1	Generic 26 32	system
T2	Task 37 63	categorizing unknown words
T3	Generic 71 77	system
T4	Method 94 122	multi-component architecture
T5	Generic 136 145	component
T6	OtherScientificTerm 192 205	unknown words
T7	Generic 240 250	components
T8	OtherScientificTerm 267 272	names
T9	OtherScientificTerm 279 294	spelling errors
T10	Generic 303 312	component
T11	Method 322 348	decision tree architecture
T12	OtherScientificTerm 401 413	unknown word
T13	Generic 421 427	system
T14	Material 459 479	live closed captions
T15	OtherScientificTerm 523 536	unknown words
R1	USED-FOR Arg1:T4 Arg2:T3
R2	USED-FOR Arg1:T7 Arg2:T8
R3	USED-FOR Arg1:T11 Arg2:T10
R4	PART-OF Arg1:T5 Arg2:T4
R5	USED-FOR Arg1:T7 Arg2:T9
R6	HYPONYM-OF Arg1:T7 Arg2:T5
R7	COREF Arg1:T13 Arg2:T3
R8	USED-FOR Arg1:T5 Arg2:T6
R9	EVALUATE-FOR Arg1:T14 Arg2:T13
R10	CONJUNCTION Arg1:T8 Arg2:T9
R11	COREF Arg1:T10 Arg2:T7
R12	USED-FOR Arg1:T1 Arg2:T2
R13	COREF Arg1:T1 Arg2:T3
--------

Annotations in ./benchmarking/input/sciERC_raw\C08-1128.ann
T1	Material 13 25	Chinese text
T2	Task 109 141	machine translation (MT) systems
T3	Task 148 150	MT
T4	Method 193 215	Chinese word segmenter
T5	Material 231 254	manually annotated data
T6	Method 288 305	word segmentation
T7	Task 339 350	translation
T8	Method 367 423	Bayesian semi-supervised Chinese word segmentation model
T9	OtherScientificTerm 442 479	monolingual and bilingual information
T10	Task 494 506	segmentation
T11	Task 522 524	MT
T12	Generic 553 559	method
T13	Method 589 598	MT system
R1	USED-FOR Arg1:T4 Arg2:T3
R2	USED-FOR Arg1:T9 Arg2:T8
R3	USED-FOR Arg1:T10 Arg2:T11
R4	COREF Arg1:T3 Arg2:T2
R5	USED-FOR Arg1:T5 Arg2:T4
R6	COREF Arg1:T6 Arg2:T4
R7	USED-FOR Arg1:T6 Arg2:T7
R8	COREF Arg1:T7 Arg2:T3
R9	USED-FOR Arg1:T8 Arg2:T10
R10	COREF Arg1:T12 Arg2:T8
R11	COMPARE Arg1:T12 Arg2:T13
R12	COREF Arg1:T11 Arg2:T7
-----------
Annotations in ./benchmarking/input/sciERC_raw\C08-2010.ann
T1	Metric 2 27	Language resource quality

Annotations in ./benchmarking/input/sciERC_raw\CVPR_2001_110_abs.ann
T1	Method 0 31	Representing images with layers
T2	Generic 51 63	applications
T3	Task 74 91	video compression
T4	Task 93 108	motion analysis
T5	Task 114 131	3D scene analysis
T6	Generic 156 164	approach
T7	OtherScientificTerm 188 194	layers
T8	Material 200 206	images
T9	OtherScientificTerm 245 257	homographies
T10	OtherScientificTerm 269 283	planar patches
T11	OtherScientificTerm 291 296	scene
T12	OtherScientificTerm 304 335	low dimensional linear subspace
T13	OtherScientificTerm 337 343	Layers
T14	Material 357 363	images
T15	OtherScientificTerm 386 394	subspace
T16	OtherScientificTerm 443 451	clusters
T17	Method 495 532	mean-shift based clustering algorithm
T18	Task 534 551	Global optimality
T19	OtherScientificTerm 580 587	regions
T20	OtherScientificTerm 631 636	noise
T21	OtherScientificTerm 681 700	subspace constraint
R1	PART-OF Arg1:T7 Arg2:T8
R2	PART-OF Arg1:T13 Arg2:T14
R3	USED-FOR Arg1:T17 Arg2:T16
R4	USED-FOR Ar

Annotations in ./benchmarking/input/sciERC_raw\CVPR_2013_10_abs.ann
T1	Generic 19 28	algorithm
T2	Task 54 100	3D geometric structure of outdoor video scenes
T3	Method 113 147	spatio-temporal video segmentation
T4	OtherScientificTerm 164 177	dynamic scene
T5	OtherScientificTerm 203 220	geometric classes
T6	Method 251 269	region-classifiers
T7	OtherScientificTerm 290 320	appearance and motion features
T8	OtherScientificTerm 409 438	segmentation hierarchy levels
T9	OtherScientificTerm 477 497	granularity a priori
T10	Generic 527 534	dataset
T11	OtherScientificTerm 538 564	geometric context of video
T12	Generic 581 587	method
T13	OtherScientificTerm 625 649	annotated outdoor videos
T14	Generic 704 711	dataset
T15	Method 726 760	semi-supervised learning framework
T16	Material 783 795	labeled data
T17	OtherScientificTerm 801 828	high confidence predictions
T18	OtherScientificTerm 843 857	unlabeled data
T19	Generic 863 869	system
T20	OtherScientificTerm 905 931	geometric context of video
T21	

Annotations in ./benchmarking/input/sciERC_raw\ECCV_2012_30_abs.ann
T1	Task 36 86	reconstructing the motion of a 3D articulated tree
T2	OtherScientificTerm 92 116	2D point correspondences
T3	OtherScientificTerm 133 147	temporal prior
T4	OtherScientificTerm 159 172	smooth motion
T5	OtherScientificTerm 201 217	trajectory basis
T6	Task 230 256	hard combinatorial problem
T7	Metric 262 277	time complexity
T8	Method 325 352	Branch and bound strategies
T9	Generic 392 402	complexity
T10	OtherScientificTerm 422 439	global optimality
T11	Generic 450 454	they
T12	Method 505 522	exhaustive search
T13	Method 594 619	compact high-pass filters
T14	Method 634 662	dynamic programming approach
T15	OtherScientificTerm 755 774	filter interactions
T16	Method 789 806	affine projection
T17	Task 815 829	reconstruction
T18	Method 838 856	estimating cameras
R1	USED-FOR Arg1:T5 Arg2:T4
R2	USED-FOR Arg1:T8 Arg2:T9
R3	COREF Arg1:T9 Arg2:T7
R4	COREF Arg1:T8 Arg2:T11
R5	COMPARE Arg1:T11 Arg2:T12
R6	USED-FOR Arg1:T16

...

Annotations in ./benchmarking/input/sciERC_raw\ICCV_2013_26_abs.ann
T1	Method 43 86	coarse-to-fine energy minimization strategy
T2	Task 91 119	semantic video segmenta-tion
T3	Generic 125 133	strategy
T4	Task 148 196	hierarchical abstraction of the supervoxel graph
T5	OtherScientificTerm 269 278	hierarchy
T6	OtherScientificTerm 335 349	coarser graphs
T7	Generic 355 363	strategy
T8	Generic 380 382	it
T9	OtherScientificTerm 433 445	finest graph
T10	Generic 447 449	It
T11	Generic 468 470	it
T12	OtherScientificTerm 499 514	energy function
T13	Method 581 610	energy minimization algorithm
T14	Method 618 628	graph cuts
T15	Method 633 651	belief propagation
T16	Generic 654 656	It
T17	Task 692 701	inference
T18	Generic 714 722	datasets
T19	OtherScientificTerm 747 773	spatio-temporal continuity
T20	Generic 827 835	strategy
T21	Method 857 880	hierarchical approaches
T22	Material 899 919	image and video data
R1	USED-FOR Arg1:T1 Arg2:T2
R2	COREF Arg1:T3 Arg2:T1
R3	USED-FOR Arg1:T4 Arg2:T3
R4	COREF A

Annotations in ./benchmarking/input/sciERC_raw\INTERSPEECH_2006_31_abs.ann
T1	Method 82 120	statistical machine learning algorithm
T2	OtherScientificTerm 150 192	smallest meaning-bearing units of language
T3	OtherScientificTerm 195 204	morphemes
T4	Generic 215 220	these
T5	Generic 271 276	tasks
T6	Task 286 315	speech and text understanding
T7	Task 317 336	machine translation
T8	Task 338 359	information retrieval
T9	Task 365 394	statistical language modeling
T10	Method 570 594	segmen-tation algorithms
T11	Task 598 633	large vocabulary speech recognition
T12	Method 640 674	statistical n-gram language models
T13	Material 766 815	ag-glutinative and morphologically rich languages
T14	Material 817 824	Finnish
T15	Material 829 837	Turk-ish
R1	HYPONYM-OF Arg1:T3 Arg2:T2
R2	USED-FOR Arg1:T1 Arg2:T2
R3	CONJUNCTION Arg1:T14 Arg2:T15
R4	HYPONYM-OF Arg1:T14 Arg2:T13
R5	HYPONYM-OF Arg1:T15 Arg2:T13
R6	HYPONYM-OF Arg1:T6 Arg2:T5
R7	HYPONYM-OF Arg1:T7 Arg2:T5
R8	CONJUNCTION Arg1:T6 Arg2:T7
R9	HYPONYM-

Annotations in ./benchmarking/input/sciERC_raw\N03-1004.ann
T1	Method 30 46	ensemble methods
T2	Task 52 68	machine learning
T3	Task 90 117	natural language processing
T4	Method 137 177	multi-strategy and multi-source approach
T5	Task 181 199	question answering
T6	Method 257 273	answering agents
T7	Method 328 344	answering agents
T8	Generic 376 386	strategies
T9	Generic 388 391	one
T10	Method 413 439	knowledge-based mechanisms
T11	Generic 449 454	other
T12	Method 465 487	statistical techniques
T13	Method 507 546	multi-level answer resolution algorithm
T14	Method 580 596	answering agents
T15	OtherScientificTerm 606 645	question, passage, and/or answer levels
T16	Method 698 725	answer resolution algorithm
T17	Generic 771 786	baseline system
T18	Metric 880 904	average precision metric
R1	COMPARE Arg1:T16 Arg2:T17
R2	USED-FOR Arg1:T1 Arg2:T2
R3	USED-FOR Arg1:T1 Arg2:T3
R4	HYPONYM-OF Arg1:T9 Arg2:T8
R5	HYPONYM-OF Arg1:T11 Arg2:T8
R6	USED-FOR Arg1:T10 Arg2:T9
R7	USED-FOR Arg1:T12 Arg2:T11
R8	

Annotations in ./benchmarking/input/sciERC_raw\P01-1004.ann
T1	OtherScientificTerm 52 65	segment order
T2	OtherScientificTerm 70 82	segmentation
T3	OtherScientificTerm 89 107	segment contiguity
T4	Task 117 126	retrieval
T5	Method 146 171	translation memory system
T6	Method 204 270	bag-of-words and segment order-sensitive string comparison methods
T7	Material 298 332	character- and word-segmented data
T8	Method 368 399	local segment contiguity models
T9	Method 418 425	N-grams
T10	Task 475 483	indexing
T11	Method 506 523	character bigrams
T12	Metric 537 555	retrieval accuracy
T13	Method 588 606	word N-gram models
T14	Method 654 674	bag-of-words methods
T15	Method 707 738	segment order-sensitive methods
T16	Metric 753 771	retrieval accuracy
R1	USED-FOR Arg1:T7 Arg2:T6
R2	USED-FOR Arg1:T11 Arg2:T10
R3	COMPARE Arg1:T14 Arg2:T15
R4	EVALUATE-FOR Arg1:T4 Arg2:T5
R5	CONJUNCTION Arg1:T8 Arg2:T6
R6	FEATURE-OF Arg1:T9 Arg2:T8
R7	CONJUNCTION Arg1:T1 Arg2:T2
R8	CONJUNCTION Arg1:T2 Arg2:T3
R9	USED-FO

Annotations in ./benchmarking/input/sciERC_raw\P06-1018.ann
T1	Method 34 56	mathematical formalism
T2	Generic 90 100	structures
T3	OtherScientificTerm 105 112	strings
T4	OtherScientificTerm 117 122	trees
T5	OtherScientificTerm 127 131	dags
T6	OtherScientificTerm 136 142	graphs
T7	Generic 162 166	them
T8	OtherScientificTerm 173 185	polarization
T9	OtherScientificTerm 210 231	elementary structures
T10	Generic 291 300	formalism
T11	Method 367 385	grammar formalisms
T12	Method 398 415	rewriting systems
T13	Method 420 439	dependency grammars
T14	OtherScientificTerm 444 447	TAG
T15	OtherScientificTerm 452 456	HPSG
T16	OtherScientificTerm 463 466	LFG
R1	HYPONYM-OF Arg1:T3 Arg2:T2
R2	HYPONYM-OF Arg1:T4 Arg2:T2
R3	HYPONYM-OF Arg1:T5 Arg2:T2
R4	HYPONYM-OF Arg1:T6 Arg2:T2
R5	CONJUNCTION Arg1:T3 Arg2:T4
R6	CONJUNCTION Arg1:T4 Arg2:T5
R7	CONJUNCTION Arg1:T5 Arg2:T6
R8	COREF Arg1:T2 Arg2:T7
R9	COREF Arg1:T1 Arg2:T10
R10	USED-FOR Arg1:T10 Arg2:T11
R11	CONJUNCTION Arg1:T15 Arg2:T16
R12	CONJUNCTION Arg

## Extract entities and relationships data into separate cols, corresponding to the .ann file

In [3]:
# Specify the directory path where your files are stored
directory_path = './benchmarking/input/sciERC_raw'

# Initialize empty lists to store the data
abstracts = []
entities = []
relationships = []

# Get the list of .ann files in the directory
ann_files = glob.glob(os.path.join(directory_path, '*.ann'))

# Iterate over each .ann file
for ann_file in ann_files:
    # Load the .ann file
    with open(ann_file, 'r') as f:
        lines = f.readlines()

    # Extract the abstract (title of .ann file)
    abstract = os.path.splitext(os.path.basename(ann_file))[0]

    # Store the annotations in a dictionary
    ann_dict = {}
    rel_dict = {}
    for line in lines:
        line_parts = line.strip().split('\t')
        if line_parts[0].startswith('T'):
            ann_id = line_parts[0]
            ann_type = line_parts[1].split()[0]
            start_offset = int(line_parts[1].split()[1])
            end_offset = int(line_parts[1].split()[2])
            annotated_text = line_parts[2]
            ann_dict[ann_id] = {
                'Annotation ID': ann_id,
                'Entity': ann_type,
                'Start Offset': start_offset,
                'End Offset': end_offset,
                'Annotated Text': annotated_text
            }
        elif line_parts[0].startswith('R'):
            rel_id = line_parts[0]
            rel_type = line_parts[1].split()[0]
            arg1_id = line_parts[1].split()[1].split(':')[1]
            arg2_id = line_parts[1].split()[2].split(':')[1]
            rel_dict[rel_id] = {
                'Relationship ID': rel_id,
                'Type': rel_type,
                'Arg1': arg1_id,
                'Arg2': arg2_id
            }

    # Append the data to the lists
    abstracts.append(abstract)
    entities.append(ann_dict)
    relationships.append(rel_dict)

# Create a DataFrame from the collected data
data = {
    'abstract_id': abstracts,
    'entities': entities,
    'relationships': relationships
}

annotations_df = pd.DataFrame(data)

In [4]:
print(len(annotations_df))
annotations_df.head()

# export to excel to view

500


,abstract_id,entities,relationships
0,A00-1024,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED..."
1,A00-2023,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP..."
2,A88-1001,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT..."
3,A88-1003,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART..."
4,A92-1010,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED..."


### Create populated_rels column, replacing entity IDs in the relationships col.

In [5]:
# Create a new column 'populated_rels' in annotations_df
annotations_df['populated_rels'] = None

# Iterate over each row in annotations_df
for index, row in annotations_df.iterrows():
    # Get the entities and relationships dictionaries for the current row
    entities_dict = row['entities']
    relationships_dict = row['relationships']
    
    # Create a new dictionary to store the populated relationships
    populated_rels_dict = {}
    
    # Iterate over each relationship in relationships_dict
    for rel_id, rel_info in relationships_dict.items():
        # Get the argument IDs
        arg1_id = rel_info['Arg1']
        arg2_id = rel_info['Arg2']
        
        # Get the corresponding annotated texts from entities_dict
        arg1_text = entities_dict[arg1_id]['Annotated Text']
        arg2_text = entities_dict[arg2_id]['Annotated Text']
        
        # Create the populated relationship string
        rel_text = f"{arg1_text} {rel_info['Type']} {arg2_text}"
        
        # Store the populated relationship in the new dictionary
        populated_rels_dict[rel_id] = {
            'Relationship ID': rel_id,
            'Rel': rel_text
        }
    
    # Assign the populated relationships dictionary to the 'populated_rels' column
    annotations_df.at[index, 'populated_rels'] = populated_rels_dict


In [6]:
print(len(annotations_df))
annotations_df.head()

# export to excel to view

500


,abstract_id,entities,relationships,populated_rels
0,A00-1024,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi..."
1,A00-2023,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki..."
2,A88-1001,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb..."
3,A88-1003,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono..."
4,A92-1010,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ..."


### Create a simplified populated_rels col, and a count_of_rels col

In [7]:
# Create a new column 'simplified_populated_rels' in annotations_df
annotations_df['simplified_populated_rels'] = None

# Iterate over each row in annotations_df
for index, row in annotations_df.iterrows():
    # Get the populated relationships dictionary for the current row
    populated_rels_dict = row['populated_rels']
    
    # Extract the relationship strings from populated_rels_dict
    rel_strings = [rel_info['Rel'] for rel_info in populated_rels_dict.values()]
    
    # Assign the simplified relationship strings to the 'simplified_populated_rels' column
    annotations_df.at[index, 'simplified_populated_rels'] = rel_strings
    
# Create a new column 'rel_count' in annotations_df
annotations_df['rel_count'] = annotations_df['simplified_populated_rels'].apply(lambda x: len(x))


In [8]:
print(len(annotations_df))
annotations_df.head()

500


,abstract_id,entities,relationships,populated_rels,simplified_populated_rels,rel_count
0,A00-1024,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13
1,A00-2023,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki...",[ranking algorithm COMPARE lattice-based appro...,9
2,A88-1001,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb...","[feedback FEATURE-OF discourse, videodisc imag...",9
3,A88-1003,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono...",[pronominal anaphora resolution module PART-OF...,3
4,A92-1010,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ...",[display of graphical information USED-FOR cog...,3


## Load .txt files (same name as corresponding .ann files)

In [9]:
# Path to the directory containing the .txt files
txt_directory = './benchmarking/input/sciERC_raw'

# Create an empty DataFrame to store the text data
text_df = pd.DataFrame(columns=['abstract_id', 'text'])

# Iterate over the rows of annotations_df
for index, row in annotations_df.iterrows():
    # Extract the abstract name
    abstract = row['abstract_id']
    
    # Construct the path to the corresponding .txt file
    txt_file_path = os.path.join(txt_directory, f'{abstract}.txt')
    
    # Load the text from the .txt file
    with open(txt_file_path, 'r') as f:
        text = f.read()
    
    # Append the abstract name and text to text_df
    text_df = pd.concat([text_df, pd.DataFrame({'abstract_id': [abstract], 'text': [text]})], ignore_index=True)
    
print(len(text_df))
text_df.head()


500


,abstract_id,text
0,A00-1024,\nThis paper introduces a system for categori...
1,A00-2023,\nThis paper presents a new approach to stati...
2,A88-1001,\n This paper describes a domain independent ...
3,A88-1003,"\n In this paper, we describe the pronominal..."
4,A92-1010,\nIn our current research into the design of ...


In [10]:
# Perform an inner join between annotations_df and text_df based on the 'Abstract' column
scierc_full = pd.merge(text_df, annotations_df, on='abstract_id', how='inner')

print(len(scierc_full))
scierc_full.head()

500


,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13
1,A00-2023,\nThis paper presents a new approach to stati...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki...",[ranking algorithm COMPARE lattice-based appro...,9
2,A88-1001,\n This paper describes a domain independent ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb...","[feedback FEATURE-OF discourse, videodisc imag...",9
3,A88-1003,"\n In this paper, we describe the pronominal...","{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono...",[pronominal anaphora resolution module PART-OF...,3
4,A92-1010,\nIn our current research into the design of ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ...",[display of graphical information USED-FOR cog...,3


In [13]:
scierc_full['word_count'] = scierc_full['text'].apply(lambda x: len(str(x).split()))

smallest_word_count = scierc_full['word_count'].min()
largest_word_count = scierc_full['word_count'].max()

print(smallest_word_count)
print(largest_word_count)

23
316


### Distinct relationship types

* Seems as if EVALUATE-FOR is not mentioned in the annotation_guideline.pdf

In [11]:
# Extract unique 'Types' from the 'relationships' column
unique_types = scierc_full['relationships'].apply(lambda x: [rel_info['Type'] for rel_info in x.values()]).explode().unique().tolist()

# Print the list of unique 'Types' prettily
for type in unique_types:
    print(type)


USED-FOR
PART-OF
HYPONYM-OF
COREF
EVALUATE-FOR
CONJUNCTION
COMPARE
FEATURE-OF
nan


# Prompt generation to detect rels
* Attempting to use similar prompt format as used in main KG_construction, for consistency.
* Aim is to benchmark against a rel-annotated dataset (scierc)

In [12]:
print(len(scierc_full))
scierc_full.head()

500


,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13
1,A00-2023,\nThis paper presents a new approach to stati...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki...",[ranking algorithm COMPARE lattice-based appro...,9
2,A88-1001,\n This paper describes a domain independent ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb...","[feedback FEATURE-OF discourse, videodisc imag...",9
3,A88-1003,"\n In this paper, we describe the pronominal...","{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono...",[pronominal anaphora resolution module PART-OF...,3
4,A92-1010,\nIn our current research into the design of ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ...",[display of graphical information USED-FOR cog...,3


#### Previous prompt used in main

In [13]:
# todo

In [14]:
# changed article to sentence.
# simplified.

# rel types:
# USED-FOR
# FEATURE-OF
# HYPONYM-OF
# PART-OF
# EVALUATE-FOR
# COMPARE
# CONJUNCTION
# COREF

# Create a function to generate the prompt based on the row values
def generate_prompt(row):

    input_text = row['text']
    
    prompt = f'''Give me a comma-separated list of relationships and coreferences detected in the provided abstract, as long as they are in this list of restricted relationship types:
    ###
    [USED-FOR, FEATURE-OF, HYPONYM-OF, PART-OF, EVALUATE-FOR, COMPARE, CONJUNCTION, COREF].
    ###
    Here are some examples of each type of relationship that may be detected:
    USED-FOR: "The TISPER system has been designed to enable many text applications, Our method models user proficiency" would become "TISPER system USED-FOR text applications".
    FEATURE-OF: "prior knowledge of the model" would become "prior knowledge FEATURE-OF model".
    HYPONYM-OF: "TUIT is a software library." would become "TUIT HYPONYM-OF software library".
    PART-OF: "We incorporate NLU module to the system." would become "NLU module PART-OF system".
    EVALUATE-FOR: "our work improves both the quality and the efficiency of entity summarization" would become "efficiency EVALUATE-FOR entity summarization".
    COMPARE: "Unlike the quantitative prior, the qualitative prior is often ignored" would become "quantitative prior COMPARE qualitative prior".
    CONJUNCTION: "obtained from human expert or knowledge base" would become "human expert CONJUNCTION knowledge base".
    COREF: "We introduce a machine reading system... The system..." would become "machine reading system COREF system".
    ###
    Desired response format: ###
    [rel1, rel2, rel3]
    ###
    Example response: ###
    ["machine reading system COREF system", "prior knowledge FEATURE-OF model", "TISPER system USED-FOR text applications"]
    ###
    The provided abstract is: ###
    {input_text}
    ###
    '''
    
    return prompt

# Add a new 'prompt' column by applying the generate_prompt function to each row
scierc_full['prompt'] = scierc_full.apply(generate_prompt, axis=1)
scierc_full['prompt'] = scierc_full['prompt'].astype(str)


In [15]:
print(len(scierc_full))
scierc_full.head()

500


,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,prompt
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13,Give me a comma-separated list of relationship...
1,A00-2023,\nThis paper presents a new approach to stati...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki...",[ranking algorithm COMPARE lattice-based appro...,9,Give me a comma-separated list of relationship...
2,A88-1001,\n This paper describes a domain independent ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb...","[feedback FEATURE-OF discourse, videodisc imag...",9,Give me a comma-separated list of relationship...
3,A88-1003,"\n In this paper, we describe the pronominal...","{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono...",[pronominal anaphora resolution module PART-OF...,3,Give me a comma-separated list of relationship...
4,A92-1010,\nIn our current research into the design of ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ...",[display of graphical information USED-FOR cog...,3,Give me a comma-separated list of relationship...


## Export the scierc_full df, with prompts, as input reference

In [16]:
# # Export DataFrame to Excel file
# excel_file_path = './benchmarking/input/scierc_full_input.xlsx'
# scierc_full.to_excel(excel_file_path, index=False)

# # Export DataFrame to pickle file
# pickle_file_path = './benchmarking/input/scierc_full_input.pickle'
# scierc_full.to_pickle(pickle_file_path)


# Exploring prompts

In [17]:
# Set the display options to show all text and split by newline
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Iterate over each row in the 'prompt' column
for prompt in scierc_full['prompt'][:1]:
    print(prompt)
    print()

Give me a comma-separated list of relationships and coreferences detected in the provided abstract, as long as they are in this list of restricted relationship types:
    ###
    [USED-FOR, FEATURE-OF, HYPONYM-OF, PART-OF, EVALUATE-FOR, COMPARE, CONJUNCTION, COREF].
    ###
    Here are some examples of each type of relationship that may be detected:
    USED-FOR: "The TISPER system has been designed to enable many text applications, Our method models user proficiency" would become "TISPER system USED-FOR text applications".
    FEATURE-OF: "prior knowledge of the model" would become "prior knowledge FEATURE-OF model".
    HYPONYM-OF: "TUIT is a software library." would become "TUIT HYPONYM-OF software library".
    PART-OF: "We incorporate NLU module to the system." would become "NLU module PART-OF system".
    EVALUATE-FOR: "our work improves both the quality and the efficiency of entity summarization" would become "efficiency EVALUATE-FOR entity summarization".
    COMPARE: "Unlike 

In [18]:
# convert to list to explore
all_prompts = scierc_full['prompt'].tolist()
print(len(all_prompts))

500


## Investigate average length of tokens

* Important for pricing forecast and LLM API restrictions

### with tiktoken

In [19]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
#encoding = tiktoken.get_encoding("cl100k_base")

In [20]:
# fns to count strings in list of prompts:

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def count_tokens_in_list(prompt_list: list, encoding_name: str) -> list:
    """Returns a list of integers representing the number of tokens in each string in the input list."""
    token_counts = []
    for prompt in prompt_list:
        num_tokens = num_tokens_from_string(prompt, encoding_name)
        token_counts.append(num_tokens)
    return token_counts


In [21]:
encoding_name = "cl100k_base" # used for gpt-3.5-turbo
token_counts = count_tokens_in_list(all_prompts, encoding_name)

In [22]:
min_tokens = min(token_counts)
max_tokens = max(token_counts)
total_tokens = sum(i for i in token_counts if isinstance(i, int))
average_tokens = total_tokens / len(all_prompts)

print(f"Number of tokens on the smallest prompt: {min_tokens}")
print(f"Number of tokens on the largest prompt: {max_tokens}")
print(f"Total number of tokens for all prompts: {total_tokens}")
print(f"Average number of tokens in all_prompts: {average_tokens}")

Number of tokens on the smallest prompt: 419
Number of tokens on the largest prompt: 839
Total number of tokens for all prompts: 273934
Average number of tokens in all_prompts: 547.868


In [23]:
# Estimating max response tokens if prompt works correctly (update to reflect prompt used)

#num_tokens_from_string("facing risk: yes. type of risk: thisis some text for a risk type.", "cl100k_base")

num_tokens_from_string(
    
    '''
["NL evaluations COMPARE Message Understanding Conferences", "evaluation methodology USED-FOR mature, practical applications", "methodology USED-FOR comparative evaluation of SLS systems", "evaluation methodology EVALUATE-FOR competing claims and identifying promising technical approaches", "methodology USED-FOR automatic evaluation of question-answering NL systems", "methodology can be used with speech or text input", "methodology FEATURE-OF heart", "DARPA SLS program PART-OF methodology implementation", "NL interfaces HYPONYM-OF speech understanding", "SLS systems HYPONYM-OF question-answering NL systems", "SLS systems PART-OF DARPA Spoken Language Systems program", "SLS systems COMPARE NL evaluations other than Message Understanding Conferences", "researchers COREF they"]
    '''
                       ,"cl100k_base" )

169

### Truncate tokens of long prompts

**** NO NEED TO TRUNCATE PROMPTS WORKING WITH scierc_full ****
**** All are below token input threshold ****

* gpt-3.5-turbo has max tokens of 4,096 tokens
* This includes prompt and response tokens combined.
* response tokens should be short due to the attempt at prompt restrictions;
    * i.e. Provide answers only in the format of <facing risk: <'yes'/'no'>. type of risk: < risk type >.> and nothing else.
* so a generous estimate of response tokens would be 100, providing gpt-3.5-turbo successfully adheres to above prompting.
* Therefore truncate prompt tokens to 3500 to be safe.

# Feed prompts into LLM

* Initially was working from a list of prompts.
* Modified to work from df and append response directly to df in new 'responses' col.
    * Allows for linking to sent_id without feeding id into prompt.

In [24]:
# Create new df to contain responses at corresponding rows.
# MODIFY TO SPECIFY AMOUNT OF INPUT.

scierc_full_responses = scierc_full.copy()
scierc_full_responses.head(1)

,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,prompt
0,A00-1024,\nThis paper introduces a system for categorizing unknown words . The system is based on a multi-component architecture where each component is responsible for identifying one class of unknown words . The focus of this paper is the components that identify names and spelling errors . Each component uses a decision tree architecture to combine multiple types of evidence about the unknown word . The system is evaluated using data from live closed captions - a genre replete with a wide variety of unknown words .\n,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Generic', 'Start Offset': 26, 'End Offset': 32, 'Annotated Text': 'system'}, 'T2': {'Annotation ID': 'T2', 'Entity': 'Task', 'Start Offset': 37, 'End Offset': 63, 'Annotated Text': 'categorizing unknown words'}, 'T3': {'Annotation ID': 'T3', 'Entity': 'Generic', 'Start Offset': 71, 'End Offset': 77, 'Annotated Text': 'system'}, 'T4': {'Annotation ID': 'T4', 'Entity': 'Method', 'Start Offset': 94, 'End Offset': 122, 'Annotated Text': 'multi-component architecture'}, 'T5': {'Annotation ID': 'T5', 'Entity': 'Generic', 'Start Offset': 136, 'End Offset': 145, 'Annotated Text': 'component'}, 'T6': {'Annotation ID': 'T6', 'Entity': 'OtherScientificTerm', 'Start Offset': 192, 'End Offset': 205, 'Annotated Text': 'unknown words'}, 'T7': {'Annotation ID': 'T7', 'Entity': 'Generic', 'Start Offset': 240, 'End Offset': 250, 'Annotated Text': 'components'}, 'T8': {'Annotation ID': 'T8', 'Entity': 'OtherScientificTerm', 'Start Offset': 267, 'End Offset': 272, 'Annotated Text': 'names'}, 'T9': {'Annotation ID': 'T9', 'Entity': 'OtherScientificTerm', 'Start Offset': 279, 'End Offset': 294, 'Annotated Text': 'spelling errors'}, 'T10': {'Annotation ID': 'T10', 'Entity': 'Generic', 'Start Offset': 303, 'End Offset': 312, 'Annotated Text': 'component'}, 'T11': {'Annotation ID': 'T11', 'Entity': 'Method', 'Start Offset': 322, 'End Offset': 348, 'Annotated Text': 'decision tree architecture'}, 'T12': {'Annotation ID': 'T12', 'Entity': 'OtherScientificTerm', 'Start Offset': 401, 'End Offset': 413, 'Annotated Text': 'unknown word'}, 'T13': {'Annotation ID': 'T13', 'Entity': 'Generic', 'Start Offset': 421, 'End Offset': 427, 'Annotated Text': 'system'}, 'T14': {'Annotation ID': 'T14', 'Entity': 'Material', 'Start Offset': 459, 'End Offset': 479, 'Annotated Text': 'live closed captions'}, 'T15': {'Annotation ID': 'T15', 'Entity': 'OtherScientificTerm', 'Start Offset': 523, 'End Offset': 536, 'Annotated Text': 'unknown words'}}","{'R1': {'Relationship ID': 'R1', 'Type': 'USED-FOR', 'Arg1': 'T4', 'Arg2': 'T3'}, 'R2': {'Relationship ID': 'R2', 'Type': 'USED-FOR', 'Arg1': 'T7', 'Arg2': 'T8'}, 'R3': {'Relationship ID': 'R3', 'Type': 'USED-FOR', 'Arg1': 'T11', 'Arg2': 'T10'}, 'R4': {'Relationship ID': 'R4', 'Type': 'PART-OF', 'Arg1': 'T5', 'Arg2': 'T4'}, 'R5': {'Relationship ID': 'R5', 'Type': 'USED-FOR', 'Arg1': 'T7', 'Arg2': 'T9'}, 'R6': {'Relationship ID': 'R6', 'Type': 'HYPONYM-OF', 'Arg1': 'T7', 'Arg2': 'T5'}, 'R7': {'Relationship ID': 'R7', 'Type': 'COREF', 'Arg1': 'T13', 'Arg2': 'T3'}, 'R8': {'Relationship ID': 'R8', 'Type': 'USED-FOR', 'Arg1': 'T5', 'Arg2': 'T6'}, 'R9': {'Relationship ID': 'R9', 'Type': 'EVALUATE-FOR', 'Arg1': 'T14', 'Arg2': 'T13'}, 'R10': {'Relationship ID': 'R10', 'Type': 'CONJUNCTION', 'Arg1': 'T8', 'Arg2': 'T9'}, 'R11': {'Relationship ID': 'R11', 'Type': 'COREF', 'Arg1': 'T10', 'Arg2': 'T7'}, 'R12': {'Relationship ID': 'R12', 'Type': 'USED-FOR', 'Arg1': 'T1', 'Arg2': 'T2'}, 'R13': {'Relationship ID': 'R13', 'Type': 'COREF', 'Arg1': 'T1', 'Arg2': 'T3'}}","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi-component architecture USED-FOR system'}, 'R2': {'Relationship ID': 'R2', 'Rel': 'components USED-FOR names'}, 'R3': {'Relationship ID': 'R3', 'Rel': 'decision tree architecture USED-FOR component'}, 'R4': {'Relationship ID': 'R4', 'Rel': 'component PART-OF multi-component arch

In [25]:
openai.api_key = os.getenv('OPENAI_KEY')

## Prompt engineering

In [26]:
# model:
GPT_MODEL = "gpt-3.5-turbo"

# Retry parameters
max_retries = 3
retry_delay = 5  # in seconds

# Initialize a new column 'responses' in the dataframe
scierc_full_responses.loc[:, 'responses'] = ''

def generate_responses(input_df):
    for idx, row in tqdm(input_df.iterrows(), total=len(input_df), desc="Generating responses"):
        try:
            query = row['prompt']

            if query is None or query.strip() == '':
                print(f"Empty prompt at index {idx}. Skipping this row.")
                continue

            retries = 0
            while retries < max_retries:
                try:
                    response = openai.ChatCompletion.create(
                        model=GPT_MODEL,
                        messages=[
                            {"role": "system", "content": "You will perform relationship and coreference extraction on the given abstract, adhering to the restricted types listed in the query"},
                            {"role": "user", "content": query}, # In general, gpt-3.5-turbo-0301 does not pay strong attention to the system message, and therefore important instructions are often better placed in a user message.
                        ],
                        temperature=0,
                        max_tokens=600,  # max tokens in response. token limit (4096) must be < largest query + message content + max_tokens
                    )

                    response_content = response['choices'][0]['message']['content']
                    print(response_content)  # Print just the message content

                    # Add the response to the 'responses' column of the dataframe
                    input_df.loc[idx, 'responses'] = response_content

                    break  # Break out of the retry loop if successful

                except Exception as e:
                    print(f"Error generating response for prompt at index {idx}: {str(e)}")
                    print(f"Query: {query}")
                    retries += 1
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)

            if retries == max_retries:
                print(f"Max retries reached for prompt at index {idx}. Skipping this row.")

            # Add a delay between requests to avoid overwhelming the API
            time.sleep(random.uniform(0.5, 1.5))

        except Exception as e:
            print(f"Error generating response for prompt at index {idx}: {str(e)}")
            print(f"Query: {query}")

## Run prompts

In [27]:
generate_responses(scierc_full_responses)

Generating responses:   0%|          | 0/500 [00:00<?, ?it/s]

["system PART-OF multi-component architecture", "component USED-FOR identifying unknown words", "names COREF spelling errors", "component FEATURE-OF decision tree architecture", "system EVALUATE-FOR live closed captions"]


Generating responses:   0%|          | 1/500 [00:05<49:41,  5.97s/it]

["approach FEATURE-OF statistical sentence generation", "alternative phrases PART-OF packed sets", "alternative phrases PART-OF forests", "representation FEATURE-OF advantages", "representation FEATURE-OF ability to represent syntactic information", "ranking algorithm FEATURE-OF efficient statistical ranking", "experimental results EVALUATE-FOR improvements over simple enumeration", "experimental results EVALUATE-FOR lattice-based approach"]


Generating responses:   0%|          | 2/500 [00:16<1:11:12,  8.58s/it]

...

Generating responses: 100%|█████████▉| 498/500 [1:34:12<00:18,  9.42s/it]

Error generating response for prompt at index 498: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 16d379e8061999a1f64a6724ff246ed5 in your message.)
Query: Give me a comma-separated list of relationships and coreferences detected in the provided abstract, as long as they are in this list of restricted relationship types:
    ###
    [USED-FOR, FEATURE-OF, HYPONYM-OF, PART-OF, EVALUATE-FOR, COMPARE, CONJUNCTION, COREF].
    ###
    Here are some examples of each type of relationship that may be detected:
    USED-FOR: "The TISPER system has been designed to enable many text applications, Our method models user proficiency" would become "TISPER system USED-FOR text applications".
    FEATURE-OF: "prior knowledge of the model" would become "prior knowledge FEATURE-OF model".
    HYPONYM-OF: "TUIT is a software library." would become "TUIT HYPO

Generating responses: 100%|█████████▉| 499/500 [1:34:58<00:20, 20.55s/it]

["automatic summarization FEATURE-OF Internet services", "MUC USED-FOR next generation Internet", "SUMMAC USED-FOR next generation Internet", "automatic summarization USED-FOR summary generation", "positive feature vectors FEATURE-OF generic summaries", "negative feature vectors FEATURE-OF generic summaries", "text model FEATURE-OF relationship between nouns and verbs", "relationship between nouns and verbs FEATURE-OF text model", "discourse segment PART-OF text model", "sentences EVALUATE-FOR user-directed summaries", "NormF EVALUATE-FOR best summary", "NormF EVALUATE-FOR fixed summary", "system COMPARE average system", "categorization task USED-FOR summary generation", "adhoc task USED-FOR summary generation"]


Generating responses: 100%|██████████| 500/500 [1:35:13<00:00, 11.43s/it]


In [28]:
scierc_full_responses.head()

,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,prompt,responses
0,A00-1024,\nThis paper introduces a system for categorizing unknown words . The system is based on a multi-component architecture where each component is responsible for identifying one class of unknown words . The focus of this paper is the components that identify names and spelling errors . Each component uses a decision tree architecture to combine multiple types of evidence about the unknown word . The system is evaluated using data from live closed captions - a genre replete with a wide variety of unknown words .\n,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Generic', 'Start Offset': 26, 'End Offset': 32, 'Annotated Text': 'system'}, 'T2': {'Annotation ID': 'T2', 'Entity': 'Task', 'Start Offset': 37, 'End Offset': 63, 'Annotated Text': 'categorizing unknown words'}, 'T3': {'Annotation ID': 'T3', 'Entity': 'Generic', 'Start Offset': 71, 'End Offset': 77, 'Annotated Text': 'system'}, 'T4': {'Annotation ID': 'T4', 'Entity': 'Method', 'Start Offset': 94, 'End Offset': 122, 'Annotated Text': 'multi-component architecture'}, 'T5': {'Annotation ID': 'T5', 'Entity': 'Generic', 'Start Offset': 136, 'End Offset': 145, 'Annotated Text': 'component'}, 'T6': {'Annotation ID': 'T6', 'Entity': 'OtherScientificTerm', 'Start Offset': 192, 'End Offset': 205, 'Annotated Text': 'unknown words'}, 'T7': {'Annotation ID': 'T7', 'Entity': 'Generic', 'Start Offset': 240, 'End Offset': 250, 'Annotated Text': 'components'}, 'T8': {'Annotation ID': 'T8', 'Entity': 'OtherScientificTerm', 'Start Offset': 267, 'End Offset': 272, 'Annotated Text': 'names'}, 'T9': {'Annotation ID': 'T9', 'Entity': 'OtherScientificTerm', 'Start Offset': 279, 'End Offset': 294, 'Annotated Text': 'spelling errors'}, 'T10': {'Annotation ID': 'T10', 'Entity': 'Generic', 'Start Offset': 303, 'End Offset': 312, 'Annotated Text': 'component'}, 'T11': {'Annotation ID': 'T11', 'Entity': 'Method', 'Start Offset': 322, 'End Offset': 348, 'Annotated Text': 'decision tree architecture'}, 'T12': {'Annotation ID': 'T12', 'Entity': 'OtherScientificTerm', 'Start Offset': 401, 'End Offset': 413, 'Annotated Text': 'unknown word'}, 'T13': {'Annotation ID': 'T13', 'Entity': 'Generic', 'Start Offset': 421, 'End Offset': 427, 'Annotated Text': 'system'}, 'T14': {'Annotation ID': 'T14', 'Entity': 'Material', 'Start Offset': 459, 'End Offset': 479, 'Annotated Text': 'live closed captions'}, 'T15': {'Annotation ID': 'T15', 'Entity': 'OtherScientificTerm', 'Start Offset': 523, 'End Offset': 536, 'Annotated Text': 'unknown words'}}","{'R1': {'Relationship ID': 'R1', 'Type': 'USED-FOR', 'Arg1': 'T4', 'Arg2': 'T3'}, 'R2': {'Relationship ID': 'R2', 'Type': 'USED-FOR', 'Arg1': 'T7', 'Arg2': 'T8'}, 'R3': {'Relationship ID': 'R3', 'Type': 'USED-FOR', 'Arg1': 'T11', 'Arg2': 'T10'}, 'R4': {'Relationship ID': 'R4', 'Type': 'PART-OF', 'Arg1': 'T5', 'Arg2': 'T4'}, 'R5': {'Relationship ID': 'R5', 'Type': 'USED-FOR', 'Arg1': 'T7', 'Arg2': 'T9'}, 'R6': {'Relationship ID': 'R6', 'Type': 'HYPONYM-OF', 'Arg1': 'T7', 'Arg2': 'T5'}, 'R7': {'Relationship ID': 'R7', 'Type': 'COREF', 'Arg1': 'T13', 'Arg2': 'T3'}, 'R8': {'Relationship ID': 'R8', 'Type': 'USED-FOR', 'Arg1': 'T5', 'Arg2': 'T6'}, 'R9': {'Relationship ID': 'R9', 'Type': 'EVALUATE-FOR', 'Arg1': 'T14', 'Arg2': 'T13'}, 'R10': {'Relationship ID': 'R10', 'Type': 'CONJUNCTION', 'Arg1': 'T8', 'Arg2': 'T9'}, 'R11': {'Relationship ID': 'R11', 'Type': 'COREF', 'Arg1': 'T10', 'Arg2': 'T7'}, 'R12': {'Relationship ID': 'R12', 'Type': 'USED-FOR', 'Arg1': 'T1', 'Arg2': 'T2'}, 'R13': {'Relationship ID': 'R13', 'Type': 'COREF', 'Arg1': 'T1', 'Arg2': 'T3'}}","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi-component architecture USED-FOR system'}, 'R2': {'Relationship ID': 'R2', 'Rel': 'components USED-FOR names'}, 'R3': {'Relationship ID': 'R3', 'Rel': 'decision tree architecture USED-FOR component'}, 'R4': {'Relationship ID': 'R4', 'Rel': 'component PART-OF multi-comp

## Compare count of rels found to annotated count.
* Very unlikely to get a hard match of the entire list of rels between simplified_populated_rels and responses.
* Also unlikely to get items to match across lists.

* Can analyse count of rels combined with manual evaluation.

* seems to have formed new rel outside of restricted rel list: OFFERS-ADVANTAGES-IN

In [29]:
scierc_full_responses['responses_rels_detected'] = scierc_full_responses['responses'].apply(lambda x: len(x.split(',')))
scierc_full_responses.head(5)

,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,prompt,responses,responses_rels_detected
0,A00-1024,\nThis paper introduces a system for categorizing unknown words . The system is based on a multi-component architecture where each component is responsible for identifying one class of unknown words . The focus of this paper is the components that identify names and spelling errors . Each component uses a decision tree architecture to combine multiple types of evidence about the unknown word . The system is evaluated using data from live closed captions - a genre replete with a wide variety of unknown words .\n,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Generic', 'Start Offset': 26, 'End Offset': 32, 'Annotated Text': 'system'}, 'T2': {'Annotation ID': 'T2', 'Entity': 'Task', 'Start Offset': 37, 'End Offset': 63, 'Annotated Text': 'categorizing unknown words'}, 'T3': {'Annotation ID': 'T3', 'Entity': 'Generic', 'Start Offset': 71, 'End Offset': 77, 'Annotated Text': 'system'}, 'T4': {'Annotation ID': 'T4', 'Entity': 'Method', 'Start Offset': 94, 'End Offset': 122, 'Annotated Text': 'multi-component architecture'}, 'T5': {'Annotation ID': 'T5', 'Entity': 'Generic', 'Start Offset': 136, 'End Offset': 145, 'Annotated Text': 'component'}, 'T6': {'Annotation ID': 'T6', 'Entity': 'OtherScientificTerm', 'Start Offset': 192, 'End Offset': 205, 'Annotated Text': 'unknown words'}, 'T7': {'Annotation ID': 'T7', 'Entity': 'Generic', 'Start Offset': 240, 'End Offset': 250, 'Annotated Text': 'components'}, 'T8': {'Annotation ID': 'T8', 'Entity': 'OtherScientificTerm', 'Start Offset': 267, 'End Offset': 272, 'Annotated Text': 'names'}, 'T9': {'Annotation ID': 'T9', 'Entity': 'OtherScientificTerm', 'Start Offset': 279, 'End Offset': 294, 'Annotated Text': 'spelling errors'}, 'T10': {'Annotation ID': 'T10', 'Entity': 'Generic', 'Start Offset': 303, 'End Offset': 312, 'Annotated Text': 'component'}, 'T11': {'Annotation ID': 'T11', 'Entity': 'Method', 'Start Offset': 322, 'End Offset': 348, 'Annotated Text': 'decision tree architecture'}, 'T12': {'Annotation ID': 'T12', 'Entity': 'OtherScientificTerm', 'Start Offset': 401, 'End Offset': 413, 'Annotated Text': 'unknown word'}, 'T13': {'Annotation ID': 'T13', 'Entity': 'Generic', 'Start Offset': 421, 'End Offset': 427, 'Annotated Text': 'system'}, 'T14': {'Annotation ID': 'T14', 'Entity': 'Material', 'Start Offset': 459, 'End Offset': 479, 'Annotated Text': 'live closed captions'}, 'T15': {'Annotation ID': 'T15', 'Entity': 'OtherScientificTerm', 'Start Offset': 523, 'End Offset': 536, 'Annotated Text': 'unknown words'}}","{'R1': {'Relationship ID': 'R1', 'Type': 'USED-FOR', 'Arg1': 'T4', 'Arg2': 'T3'}, 'R2': {'Relationship ID': 'R2', 'Type': 'USED-FOR', 'Arg1': 'T7', 'Arg2': 'T8'}, 'R3': {'Relationship ID': 'R3', 'Type': 'USED-FOR', 'Arg1': 'T11', 'Arg2': 'T10'}, 'R4': {'Relationship ID': 'R4', 'Type': 'PART-OF', 'Arg1': 'T5', 'Arg2': 'T4'}, 'R5': {'Relationship ID': 'R5', 'Type': 'USED-FOR', 'Arg1': 'T7', 'Arg2': 'T9'}, 'R6': {'Relationship ID': 'R6', 'Type': 'HYPONYM-OF', 'Arg1': 'T7', 'Arg2': 'T5'}, 'R7': {'Relationship ID': 'R7', 'Type': 'COREF', 'Arg1': 'T13', 'Arg2': 'T3'}, 'R8': {'Relationship ID': 'R8', 'Type': 'USED-FOR', 'Arg1': 'T5', 'Arg2': 'T6'}, 'R9': {'Relationship ID': 'R9', 'Type': 'EVALUATE-FOR', 'Arg1': 'T14', 'Arg2': 'T13'}, 'R10': {'Relationship ID': 'R10', 'Type': 'CONJUNCTION', 'Arg1': 'T8', 'Arg2': 'T9'}, 'R11': {'Relationship ID': 'R11', 'Type': 'COREF', 'Arg1': 'T10', 'Arg2': 'T7'}, 'R12': {'Relationship ID': 'R12', 'Type': 'USED-FOR', 'Arg1': 'T1', 'Arg2': 'T2'}, 'R13': {'Relationship ID': 'R13', 'Type': 'COREF', 'Arg1': 'T1', 'Arg2': 'T3'}}","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi-component architecture USED-FOR system'}, 'R2': {'Relationship ID': 'R2', 'Rel': 'components USED-FOR names'}, 'R3': {'Relationship ID': 'R3', 'Rel': 'decision tree architecture USED-FOR component'}, 'R4': {'Relationship ID': 'R4', 'Rel': 'comp

### Quantitative comparisons

In [30]:
rel_count_sum = scierc_full_responses['rel_count'].sum()
responses_rels_detected_sum = scierc_full_responses['responses_rels_detected'].sum()

print("Sum of rel_count:", rel_count_sum)
print("Sum of responses_rels_detected:", responses_rels_detected_sum)


Sum of rel_count: 6394
Sum of responses_rels_detected: 3958


In [31]:
# Define the target texts
target_texts = ['USED-FOR', 'FEATURE-OF', 'HYPONYM-OF', 'PART-OF', 'EVALUATE-FOR', 'COMPARE', 'CONJUNCTION', 'COREF']

# Initialize the dictionary
rel_summary = {text: 0 for text in target_texts}

# Count the occurrences of each target text
for row in scierc_full_responses['responses']:
    phrases = row.split(', ')
    for text in target_texts:
        for phrase in phrases:
            if text in phrase:
                rel_summary[text] += 1

# Print the summary statistics
print(rel_summary)

# Print the sum of values
print("Sum of Values:", sum(rel_summary.values))

{'USED-FOR': 712, 'FEATURE-OF': 902, 'HYPONYM-OF': 158, 'PART-OF': 583, 'EVALUATE-FOR': 574, 'COMPARE': 404, 'CONJUNCTION': 15, 'COREF': 136}
Sum of Values: 3484


In [32]:
# Above implies that there are some rels generated outside of the restricted options;
# Or, may be slight mis-matches with letter-casing, lack of hyphen, etc.

# Final export for manual verification

In [33]:
# # export to excel for manual verification:
# scierc_full_responses.to_excel("./benchmarking/results/scierc_rel_detection_responses.xlsx")

# # export to pickle in case of future use.
# scierc_full_responses.to_pickle("./benchmarking/results/scierc_rel_detection_responses.pkl")

# Re-import pickle file for exploratory analysis
* Used in V2 COPY (used when writing dissertation):

In [100]:
# Load the pickle file
reimport_scierc_results = pd.read_pickle("./benchmarking/results/scierc_rel_detection_responses.pkl")
print(len(reimport_scierc_results))
reimport_scierc_results.head(1)

500


,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,prompt,responses,responses_rels_detected
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13,Give me a comma-separated list of relationship...,"[""system PART-OF multi-component architecture""...",5


### remove rows where abstract ID had missing annotation file (C88-2132), or no rels detected (other 2)

In [101]:

# List of abstract values to be excluded
excluded_abstracts = ['IJCAI_2013_4_abs', 'AAAI_2008_262_abs', 'C88-2132']

# Filter out rows with excluded abstract values
reimport_scierc_results = reimport_scierc_results[~reimport_scierc_results['abstract_id'].isin(excluded_abstracts)]
print(len(reimport_scierc_results))
reimport_scierc_results.head(1)

497


,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,prompt,responses,responses_rels_detected
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13,Give me a comma-separated list of relationship...,"[""system PART-OF multi-component architecture""...",5


## Add comparison cols:
1) exact rel in annotated but not in responses.
2) corresponding count of rels.
3) exact rel in responses but not in annotated.
4) corresponding count of rels. 
5) exact rel in responses and annotations.
6) corresponding count of rels.

In [102]:
# Remove punctuation contained in most responses
reimport_scierc_results['responses'] = reimport_scierc_results['responses'].str.replace('[\[\]"]', '', regex=True)

# Trim leading and trailing spaces/newlines
# reimport_scierc_results['responses'] = reimport_scierc_results['responses'].apply(lambda x: [item.strip() for item in x])

# Display the updated DataFrame
reimport_scierc_results.head(1)


,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,prompt,responses,responses_rels_detected
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13,Give me a comma-separated list of relationship...,"system PART-OF multi-component architecture, c...",5


In [103]:
# determine if comparison cols are list of items or string.
# Assuming the column name is 'col_name'
col_names = ('simplified_populated_rels', 'responses')

for col in col_names:
    col_name = col
    # Check the type of values in the column
    value_type = type(reimport_scierc_results.loc[0, col_name])

    # Check if the column contains a list
    if value_type == list:
        print(f"The '{col_name}' column contains a list of items.")
    else:
        print(f"The '{col_name}' column contains strings.")

# output must be same format for accurate comparison.

The 'simplified_populated_rels' column contains a list of items.
The 'responses' column contains strings.


In [104]:
# Assuming the column name is 'responses'
col_name = 'responses'

# Convert the string to a list of items
reimport_scierc_results[col_name] = reimport_scierc_results[col_name].str.split(',')

# Check the updated type of values in the column
value_type = type(reimport_scierc_results.loc[0, col_name])

# Check if the column contains a list
if value_type == list:
    print(f"The '{col_name}' column has been converted to a list of items.")
else:
    print(f"Error: The '{col_name}' column could not be converted to a list.")
reimport_scierc_results.head(1)


The 'responses' column has been converted to a list of items.


In [105]:
# strip
reimport_scierc_results['responses'] = reimport_scierc_results['responses'].apply(lambda x: [item.strip() for item in x])
reimport_scierc_results.head(1)

In [107]:
# Step 2: Create 'rel_annotated_but_not_detected' column
reimport_scierc_results['rel_annotated_but_not_detected'] = reimport_scierc_results.apply(lambda row: [rel for rel in row['simplified_populated_rels'] if rel not in row['responses']], axis=1)
reimport_scierc_results['rel_annotated_but_not_detected_count'] = reimport_scierc_results['rel_annotated_but_not_detected'].apply(len)

# Step 3: Create 'rel_detected_but_not_annotated' column
reimport_scierc_results['rel_detected_but_not_annotated'] = reimport_scierc_results.apply(lambda row: [rel for rel in row['responses'] if rel not in row['simplified_populated_rels']], axis=1)
reimport_scierc_results['rel_detected_but_not_annotated_count'] = reimport_scierc_results['rel_detected_but_not_annotated'].apply(len)

# Step 4: Create 'annotated_rel_detected' column
reimport_scierc_results['annotated_rel_detected'] = reimport_scierc_results.apply(lambda row: [rel for rel in row['responses'] if rel in row['simplified_populated_rels']], axis=1)
reimport_scierc_results['annotated_rel_detected_count'] = reimport_scierc_results['annotated_rel_detected'].apply(len)


In [109]:
reimport_scierc_results.head()

,abstract_id,text,entities,relationships,populated_rels,simplified_populated_rels,rel_count,prompt,responses,responses_rels_detected,rel_annotated_but_not_detected,rel_annotated_but_not_detected_count,rel_detected_but_not_annotated,rel_detected_but_not_annotated_count,annotated_rel_detected,annotated_rel_detected_count
0,A00-1024,\nThis paper introduces a system for categori...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'multi...","[multi-component architecture USED-FOR system,...",13,Give me a comma-separated list of relationship...,"[system PART-OF multi-component architecture, ...",5,"[multi-component architecture USED-FOR system,...",13,"[system PART-OF multi-component architecture, ...",5,[],0
1,A00-2023,\nThis paper presents a new approach to stati...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Gene...","{'R1': {'Relationship ID': 'R1', 'Type': 'COMP...","{'R1': {'Relationship ID': 'R1', 'Rel': 'ranki...",[ranking algorithm COMPARE lattice-based appro...,9,Give me a comma-separated list of relationship...,[approach FEATURE-OF statistical sentence gene...,8,[ranking algorithm COMPARE lattice-based appro...,9,[approach FEATURE-OF statistical sentence gene...,8,[],0
2,A88-1001,\n This paper describes a domain independent ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'FEAT...","{'R1': {'Relationship ID': 'R1', 'Rel': 'feedb...","[feedback FEATURE-OF discourse, videodisc imag...",9,Give me a comma-separated list of relationship...,[natural language interface USED-FOR database ...,7,"[feedback FEATURE-OF discourse, videodisc imag...",8,[multimedia answers FEATURE-OF videodisc image...,6,[natural language interface USED-FOR database ...,1
3,A88-1003,"\n In this paper, we describe the pronominal...","{'T1': {'Annotation ID': 'T1', 'Entity': 'Meth...","{'R1': {'Relationship ID': 'R1', 'Type': 'PART...","{'R1': {'Relationship ID': 'R1', 'Rel': 'prono...",[pronominal anaphora resolution module PART-OF...,3,Give me a comma-separated list of relationship...,[pronominal anaphora resolution module PART-OF...,7,[module COREF pronominal anaphora resolution m...,2,"[anaphora resolution HYPONYM-OF theories, blac...",6,[pronominal anaphora resolution module PART-OF...,1
4,A92-1010,\nIn our current research into the design of ...,"{'T1': {'Annotation ID': 'T1', 'Entity': 'Task...","{'R1': {'Relationship ID': 'R1', 'Type': 'USED...","{'R1': {'Relationship ID': 'R1', 'Rel': 'displ...",[display of graphical information USED-FOR cog...,3,Give me a comma-separated list of relationship...,[natural language generation USED-FOR interact...,3,[display of graphical information USED-FOR cog...,2,[graphical information FEATURE-OF cognitively ...,2,[natural language generation USED-FOR interact...,1


### Export scierc_full_responses with additional analyses

In [110]:
reimport_scierc_results.to_excel("./benchmarking/results/V2_scierc_rel_detection_responses.xlsx")


## iterate over input annotations to get distribution of relation types

In [79]:
# Assuming 'reimport_scierc_results' is your DataFrame
simplified_populated_rels_list = [str(item) for item in reimport_scierc_results['simplified_populated_rels']]

In [25]:
# Define the target texts
target_texts = ['USED-FOR', 'FEATURE-OF', 'HYPONYM-OF', 'PART-OF', 'EVALUATE-FOR', 'COMPARE', 'CONJUNCTION', 'COREF']

# Initialize the dictionary
rel_summary = {text: 0 for text in target_texts}

# Count the occurrences of each target text
for row in simplified_populated_rels_list:
    phrases = row.split(', ')
    for text in target_texts:
        for phrase in phrases:
            if text in phrase:
                rel_summary[text] += 1

# Print the summary statistics
print(rel_summary)

# Print the sum of values
print("Sum of Values:", sum(rel_summary.values()))


{'USED-FOR': 2437, 'FEATURE-OF': 264, 'HYPONYM-OF': 477, 'PART-OF': 270, 'EVALUATE-FOR': 454, 'COMPARE': 234, 'CONJUNCTION': 583, 'COREF': 1675}
Sum of Values: 6394


## Explore 10 randomly selected samples where ChatGPT detected more rels than was annotated

In [27]:
seedset = 3409578

# Filter the DataFrame based on the condition
responses_more_than_annotated = reimport_scierc_results[reimport_scierc_results['responses_rels_detected'] > reimport_scierc_results['rel_count']]
responses_less_than_annotated = reimport_scierc_results[reimport_scierc_results['responses_rels_detected'] < reimport_scierc_results['rel_count']]


# Randomly select 10 samples from the filtered DataFrame
CGPT_detected_more = responses_more_than_annotated.sample(n=10, random_state=random.seed(seedset))
CGPT_detected_less = responses_less_than_annotated.sample(n=10, random_state=random.seed(seedset))

# Export the DataFrames to an Excel file with two sheets
with pd.ExcelWriter('./benchmarking/results/qual_analysis_scierc.xlsx') as writer:
    CGPT_detected_more.to_excel(writer, sheet_name='CGPT_detected_more')
    CGPT_detected_less.to_excel(writer, sheet_name='CGPT_detected_less')